# Context

In the recent stress test of our application GlugiZP2000, we had severe issues with data quality and program failures. After an initial inspection of our code review team, we found several spot in the application that can lead to race conditions.

Race conditions are bad because they lead to weird behavior in multi-user applications. Wikipedia defines [Race Conditions](https://en.wikipedia.org/wiki/Race_condition) as follows:

> A race condition or race hazard is the behavior of an electronics, software, or other system where the output is dependent on the sequence or timing of other uncontrollable events. It becomes a bug when events do not happen in the order the programmer intended.



# Idea
Some of the occurred race conditions could be statically identified by our software expert John Doe. In the following analysis, we use [jQAssistant](https://jqassistant.org/) and the [Neo4j](https://neo4j.com/) graph database to spot even more possible problems with the help of structural code analysis. For this, we read in the complete Java code of GlugiZP2000 into the Neo4j database.

# Analysis
With the following Cypher queries, we can spot some kind of race conditions that are declared public and not static fields and are written by some methods.

In [1]:
%load_ext cypher

In [2]:
%%cypher
MATCH (c:Class)-[:DECLARES]->(f:Field)<-[w:WRITES]-(m:Method)
WHERE 
    EXISTS(f.static) AND NOT EXISTS(f.final)
RETURN 
    c.name as InClass, 
    m.name as theMethod, 
    w.lineNumber as writesInLine, 
    f.name as toStaticField

3 rows affected.


InClass,theMethod,writesInLine,toStaticField
OwnerController,processFindForm,110,ownersIndexes
OwnerController,processFindForm,110,ownersIndexes
BrokenSingleton,getInstance,11,INSTANCE


# Conclusion
There are some possible race conditions regarding static, written Java field references. Our suggestion is to fix these immediately.